In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
%cd /content/drive/My\ Drive/Natural Language Processing/week3

/content/drive/My Drive/Natural Language Processing/week3


In [ ]:
from setup_google_colab import setup_starspace

In [ ]:
setup_starspace()

In [ ]:
!ls

boost_1_63_0			       __pycache__
boost_1_63_0.zip		       setup_google_colab.py
boost_1_63_0.zip.1		       Starspace
data				       util.py
GoogleNews-vectors-negative300.bin.gz  week3-Embeddings.ipynb
grader.py


In [ ]:
!ls Starspace

3rdparty	    dict.o	  makefile	     normalize.o  src
args.o		    doc_data.o	  makefile_compress  parser.o	  starspace
CODE_OF_CONDUCT.md  doc_parser.o  makefile_py	     proj.o	  starspace.o
CONTRIBUTING.md     examples	  model.o	     python	  utils.o
data.o		    LICENSE.md	  MVS		     README.md


# Find duplicate questions on StackOverflow by their embeddings

In this assignment you will learn how to calculate a similarity for pieces of text. Using this approach you will know how to find duplicate questions from [StackOverflow](https://stackoverflow.com).

### Libraries

In this task you will you will need the following libraries:
- [StarSpace](https://github.com/facebookresearch/StarSpace) — a general-purpose model for efficient learning of entity embeddings from Facebook
- [Gensim](https://radimrehurek.com/gensim/) — a tool for solving various NLP-related tasks (topic modeling, text representation, ...)
- [Numpy](http://www.numpy.org) — a package for scientific computing.
- [scikit-learn](http://scikit-learn.org/stable/index.html) — a tool for data mining and data analysis.
- [Nltk](http://www.nltk.org) — a platform to work with human language data.

### Data

The following cell will download all data required for this assignment into the folder `week3/data`.

In [ ]:
import numpy as np

In [ ]:
import sys
sys.path.append("..")
from common.download_utils import download_week3_resources

download_week3_resources()

### Grading
We will create a grader instace below and use it to collect your answers. Note that these outputs will be stored locally inside grader and will be uploaded to platform only after running submiting function in the last part of this assignment. If you want to make partial submission, you can run that cell any time you want.

In [ ]:
from grader import Grader

In [ ]:
grader = Grader()

## Word embedding

To solve the problem, you will use two different models of embeddings:

 - [Pre-trained word vectors](https://code.google.com/archive/p/word2vec/) from Google which were trained on a part of Google News dataset (about 100 billion words). The model contains 300-dimensional vectors for 3 million words and phrases. `GoogleNews-vectors-negative300.bin.gz` will be downloaded in `download_week3_resources()`.
 - Representations using StarSpace on StackOverflow data sample. You will need to train them from scratch.

It's always easier to start with pre-trained embeddings. Unpack the pre-trained Goggle's vectors and upload them using the function [KeyedVectors.load_word2vec_format](https://radimrehurek.com/gensim/models/keyedvectors.html) from gensim library with the parameter *binary=True*. If the size of the embeddings is larger than the avaliable memory, you could load only a part of the embeddings by defining the parameter *limit* (recommended: 500000).

In [ ]:
!ls

data				       grader.py    util.py
GoogleNews-vectors-negative300.bin.gz  __pycache__  week3-Embeddings.ipynb


In [ ]:
import gensim
from gensim.models import KeyedVectors
import warnings
import pandas as pd

In [ ]:
warnings.filterwarnings("default", category=FutureWarning)

In [ ]:
wv_embeddings = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz',binary=True)             
                            

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
print(type(wv_embeddings))
wv_model_list = list( wv_embeddings.wv.vocab )
print('No. of different words: ',len(wv_model_list))

<class 'gensim.models.keyedvectors.Word2VecKeyedVectors'>
No. of different words:  3000000


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


In [ ]:
wv_model_dict = wv_embeddings.wv.vocab
print(type(wv_model_dict))
print('No. of items in the dictionary: ', len(wv_model_dict))

<class 'dict'>
No. of items in the dictionary:  3000000


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


In [ ]:
# Code-check
wv_model_dict['Australia']

In [ ]:
print(type(wv_embeddings))

<class 'gensim.models.keyedvectors.Word2VecKeyedVectors'>


### How to work with Google's word2vec embeddings?

Once you have loaded the representations, make sure you can access them. First, you can check if the loaded embeddings contain a word:
    
    'word' in wv_embeddings
    
Second, to get the corresponding embedding you can use the square brackets:

    wv_embeddings['word']
 
### Checking that the embeddings are correct 
 
To prevent any errors during the first stage, we can check that the loaded embeddings are correct. You can call the function *check_embeddings*, implemented below, which runs 3 tests:
1. Find the most similar word for provided "positive" and "negative" words.
2. Find which word from the given list doesn’t go with the others.
3. Find the most similar word for the provided one.

In the right case the function will return the string *These embeddings look good*. Othervise, you need to validate the previous steps.

In [ ]:
# Checking
if 'word' in wv_embeddings:                     # 'word' exist in the wv_embeddings
  print(wv_embeddings['word'].shape)            # Gives the vector of dim 300
  print(type(wv_embeddings['word']))

# wv_embedddings['word'] gives the ndarray of 300 dimensions.

(300,)
<class 'numpy.ndarray'>


In [ ]:
def check_embeddings(embeddings):
    error_text = "Something wrong with your embeddings ('%s test isn't correct)."
    most_similar = embeddings.most_similar(positive=['woman', 'king'], negative=['man'])
    if len(most_similar) < 1 or most_similar[0][0] != 'queen':
        return error_text % "Most similar"

    doesnt_match = embeddings.doesnt_match(['breakfast', 'cereal', 'dinner', 'lunch'])
    if doesnt_match != 'cereal':
        return error_text % "Doesn't match"
    
    most_similar_to_given = embeddings.most_similar_to_given('music', ['water', 'sound', 'backpack', 'mouse'])
    if most_similar_to_given != 'sound':
        return error_text % "Most similar to given"
    
    return "These embeddings look good."

In [ ]:
print(check_embeddings(wv_embeddings))               # returns these embeddings looks good

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


These embeddings look good.


/usr/local/lib/python3.6/dist-packages/gensim/models/keyedvectors.py:895: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


In [ ]:
# Code-check
most_similar = wv_embeddings.most_similar(positive=['woman', 'king'], negative=['man'])
print(most_similar)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('queen', 0.7118192911148071), ('monarch', 0.6189674139022827), ('princess', 0.5902431011199951), ('crown_prince', 0.5499460697174072), ('prince', 0.5377321243286133), ('kings', 0.5236844420433044), ('Queen_Consort', 0.5235945582389832), ('queens', 0.518113374710083), ('sultan', 0.5098593235015869), ('monarchy', 0.5087411999702454)]


## From word to text embeddings

**Task 1 (Question2Vec).** Usually, we have word-based embeddings, but for the task we need to create a representation for the whole question. It could be done in different ways. In our case we will use a **mean** of all word vectors in the question. Now you need to implement the function *question_to_vec*, which calculates the question representation described above. This function should work with the input text as is without any preprocessing.

Note that there could be words without the corresponding embeddings. In this case, you can just skip these words and don't take them into account during calculating the result. If the question doesn't contain any known word with embedding, the function should return a zero vector.

In [ ]:
import numpy as np

In [ ]:
# Code-check if '' lies in the w_embedding
if '' in wv_embeddings:
  print('yes')
else:
  print('No')

No


In [ ]:
def question_to_vec(question, embeddings, dim=300):
    """
        question: a string
        embeddings: dict where the key is a word and a value is its' embedding
        dim: size of the representation

        result: vector representation for the question
    """
    count = int(0)                      # initialize the count to determine the no. of words actually present in the embeddings.
    embed = np.zeros(dim)               # intialize the embedd of zeros vector of dim=300 


    if len(question)==0:
      return embed

    else:
      for wrd in question.split():

        if wrd in embeddings:
          count+=1
          embed += embeddings[wrd]
      

    if(count!=0):
      return embed/count

    return embed                     # return the vector



In [ ]:
# Code-check
ans = question_to_vec("Coffee is the best", wv_embeddings, dim=300)
print(type(ans))
print(ans)

<class 'numpy.ndarray'>
[-0.02411652 -0.06079102  0.01098633  0.18429565 -0.00897217  0.03004456
  0.0612793  -0.07623291  0.07035828  0.06880188  0.01300049 -0.03723145
  0.16325378  0.02246094  0.03848267  0.14819336 -0.04292297  0.09915924
 -0.02153778 -0.06143188 -0.02140045  0.09448242 -0.09234619 -0.06971931
  0.02587891  0.10125732 -0.11755371  0.15942383  0.06432724  0.0222168
 -0.02764893 -0.09384155 -0.02325439  0.02292633  0.02191162  0.09529114
 -0.1137085  -0.12695312 -0.0065918  -0.01907349  0.00946045 -0.03637695
 -0.05911255  0.06530762 -0.01963806  0.05409622 -0.04980469  0.01809692
  0.09417725  0.0559082   0.01116943  0.15635681 -0.0160675  -0.06713867
  0.00738525  0.09130859  0.03500366 -0.01604462  0.00242615 -0.09402466
 -0.05432129  0.1114502  -0.11883545 -0.02423096  0.02401733 -0.07125092
 -0.01751709  0.06744385 -0.04933548  0.01745605  0.10171509 -0.03863364
  0.1661377   0.04299927 -0.10430908 -0.11122131  0.05523682  0.02716064
  0.04013062  0.05262756 -0.

In [ ]:
# Code-check
sent='word'
for wrd in sent.split():
  if wrd in wv_embeddings:
    print('yes')
  else:
    print('no')

  print(wrd)

yes
word


To check the basic correctness of your implementation, run the function *question_to_vec_tests*.

In [ ]:
def question_to_vec_tests():
    if (np.zeros(300) != question_to_vec('', wv_embeddings)).any():
        return "You need to return zero vector for empty question."
    if (np.zeros(300) != question_to_vec('thereisnosuchword', wv_embeddings)).any():
        return "You need to return zero vector for the question, which consists only unknown words."
    if (wv_embeddings['word'] != question_to_vec('word', wv_embeddings)).any():
        return "You need to check the corectness of your function."
    if ((wv_embeddings['I'] + wv_embeddings['am']) / 2 != question_to_vec('I am', wv_embeddings)).any():
        return "Your function should calculate a mean of word vectors."
    if (wv_embeddings['word'] != question_to_vec('thereisnosuchword word', wv_embeddings)).any():
        return "You should not consider words which embeddings are unknown."
    return "Basic tests are passed."

In [ ]:
print(question_to_vec_tests())

Basic tests are passed.


You can submit embeddings for the questions from the file *test_embeddings.tsv* to earn the points. In this task you don't need to transform the text of a question somehow.

In [ ]:
data = pd.read_csv('./data/test_embeddings.tsv', sep='\t', header=None,)
data.head()

,0
0,Play Framework 1.2.5 : OutOfMemoryError occure...
1,Navigating between several views in iphone
2,A strange phenomenon of vector member in a class
3,Formula to auto change all calendar hours if I...
4,Is there a way to redirect the browser from th...


In [ ]:
import nltk
nltk.download('stopwords')
from util import array_to_string

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
question2vec_result = []
for question in open('data/test_embeddings.tsv'):
    question = question.strip()
    answer = question_to_vec(question, wv_embeddings)
    question2vec_result = np.append(question2vec_result, answer)

grader.submit_tag('Question2Vec', array_to_string(question2vec_result))

Current answer for task Question2Vec is: 0.019293891059027776
-0.028727213541666668
0.046056111653645836
0.08525933159722222
0.02430555555555...


Now we have a method to create a representation of any sentence and we are ready for the first evaluation. So, let's check how well our solution (Google's vectors + *question_to_vec*) will work.

## Evaluation of text similarity

We can imagine that if we use good embeddings, the cosine similarity between the duplicate sentences should be less than for the random ones. Overall, for each pair of duplicate sentences we can generate *R* random negative examples and find out the position of the correct duplicate.  

For example, we have the question *"Exceptions What really happens"* and we are sure that another question *"How does the catch keyword determine the type of exception that was thrown"* is a duplicate. But our model doesn't know it and tries to find out the best option also among questions like *"How Can I Make These Links Rotate in PHP"*, *"NSLog array description not memory address"* and *"PECL_HTTP not recognised php ubuntu"*. The goal of the model is to rank all these 4 questions (1 *positive* and *R* = 3 *negative*) in the way that the correct one is in the first place.

However, it is unnatural to count on that the best candidate will be always in the first place. So let us consider the place of the best candidate in the sorted list of candidates and formulate a metric based on it. We can fix some *K* — a reasonalble number of top-ranked elements and *N* — a number of queries (size of the sample).

### Hits@K

The first simple metric will be a number of correct hits for some *K*:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [dup_i \in topK(q_i)]$$

where $q_i$ is the i-th query, $dup_i$ is its duplicate, $topK(q_i)$ is the top K elements of the ranked sentences provided by our model and the operation $[dup_i \in topK(q_i)]$ equals 1 if the condition is true and 0 otherwise (more details about this operation could be found [here](https://en.wikipedia.org/wiki/Iverson_bracket)).


### DCG@K
The second one is a simplified [DCG metric](https://en.wikipedia.org/wiki/Discounted_cumulative_gain):

$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank_{dup_i})}\cdot[rank_{dup_i} \le K] $$

where $rank_{dup_i}$ is a position of the duplicate in the sorted list of the nearest sentences for the query $q_i$. According to this metric, the model gets a higher reward for a higher position of the correct answer. If the answer does not appear in topK at all, the reward is zero. 

### Evaluation examples

Let's calculate the described metrics for the toy example introduced above. In this case $N$ = 1 and the correct candidate for $q_1$ is *"How does the catch keyword determine the type of exception that was thrown"*. Consider the following ranking of the candidates:
1. *"How Can I Make These Links Rotate in PHP"*
2. *"How does the catch keyword determine the type of exception that was thrown"*
3. *"NSLog array description not memory address"*
4. *"PECL_HTTP not recognised php ubuntu"*

Using the ranking above, calculate *Hits@K* metric for *K = 1, 2, 4*: 
 
- [K = 1] $\text{Hits@1} = \frac{1}{1}\sum_{i=1}^1 \, [dup_i \in top1(q_i)] = [dup_1 \in top1(q_1)] = 0$ because the correct answer doesn't appear in the *top1* list.
- [K = 2] $\text{Hits@2} = \frac{1}{1}\sum_{i=1}^1 \, [dup_i \in top2(q_i)] = [dup_1 \in top2(q_1)] = 1$ because $rank_{dup_1} = 2$.
- [K = 4] $\text{Hits@4} = \frac{1}{1}\sum_{i=1}^1 \, [dup_i \in top4(q_i)] = [dup_1 \in top4(q_1)] = 1$

Using the ranking above, calculate *DCG@K* metric for *K = 1, 2, 4*:

- [K = 1] $\text{DCG@1} = \frac{1}{1} \sum_{i=1}^1\frac{1}{\log_2(1+rank_{dup_i})}\cdot[rank_{dup_i} \le 1] = \frac{1}{\log_2(1+rank_{dup_i})}\cdot[rank_{dup_i} \le 1] = 0$ because the correct answer doesn't appear in the top1 list.
- [K = 2] $\text{DCG@2} = \frac{1}{1} \sum_{i=1}^1\frac{1}{\log_2(1+rank_{dup_i})}\cdot[rank_{dup_i} \le 2] = \frac{1}{\log_2{3}}$, because $rank_{dup_1} = 2$.
- [K = 4] $\text{DCG@4} = \frac{1}{1} \sum_{i=1}^1\frac{1}{\log_2(1+rank_{dup_i})}\cdot[rank_{dup_i} \le 4] = \frac{1}{\log_2{3}}$.


**Tasks 2 and 3 (HitsCount and DCGScore).** Implement the functions *hits_count* and *dcg_score* as described above. Each function has two arguments: *dup_ranks* and *k*. *dup_ranks* is a list which contains *values of ranks* of duplicates. For example, *dup_ranks* is *[2]* for the example provided above.

In [ ]:
def hits_count(dup_ranks, k):
    """
        dup_ranks: list of duplicates' ranks; one rank per question; 
                   length is a number of questions which we are looking for duplicates; 
                   rank is a number from 1 to len(candidates of the question); 
                   e.g. [2, 3] means that the first duplicate has the rank 2, the second one — 3.
        k: number of top-ranked elements (k in Hits@k metric)

        result: return Hits@k value for current ranking
    """
    count=int(0)
    length = len(dup_ranks)

    for rank in dup_ranks:
          if(k>=rank):
            count+=1
          else:
            count+=0
    return count/length


Test your code on the tiny examples:

In [ ]:
# Code-check
answers = ["How does the catch keyword determine the type of exception that was thrown", 
               "Convert Google results object (pure js) to Python object"]
candidates_ranking = [["How Can I Make These Links Rotate in PHP", 
                           "How does the catch keyword determine the type of exception that was thrown"], 
                          ["Convert Google results object (pure js) to Python object",
                           "WPF- How to update the changes in list item of a list"]]
dup_ranks = [candidates_ranking[i].index(answers[i]) + 1 for i in range(len(answers))]
print(dup_ranks)

[2, 1]


In [ ]:
def test_hits():
    # *Evaluation example*
    # answers — dup_i
    answers = ["How does the catch keyword determine the type of exception that was thrown"]
    
    # candidates_ranking — the ranked sentences provided by our model
    candidates_ranking = [["How Can I Make These Links Rotate in PHP", 
                           "How does the catch keyword determine the type of exception that was thrown",
                           "NSLog array description not memory address",
                           "PECL_HTTP not recognised php ubuntu"]]
    # dup_ranks — position of the dup_i in the list of ranks +1
    dup_ranks = [candidates_ranking[i].index(answers[i]) + 1 for i in range(len(answers))]
    
    # correct_answers — the expected values of the result for each k from 1 to 4
    correct_answers = [0, 1, 1, 1]
    for k, correct in enumerate(correct_answers, 1):
        if not np.isclose(hits_count(dup_ranks, k), correct):
            return "Check the function."
    
    # Other tests
    answers = ["How does the catch keyword determine the type of exception that was thrown", 
               "Convert Google results object (pure js) to Python object"]
    
    # The first test: both duplicates on the first position in ranked list
    candidates_ranking = [["How does the catch keyword determine the type of exception that was thrown",
                           "How Can I Make These Links Rotate in PHP"], 
                          ["Convert Google results object (pure js) to Python object",
                           "WPF- How to update the changes in list item of a list"]]
    dup_ranks = [candidates_ranking[i].index(answers[i]) + 1 for i in range(len(answers))]
    correct_answers = [1, 1]
    for k, correct in enumerate(correct_answers, 1):
        if not np.isclose(hits_count(dup_ranks, k), correct):
            return "Check the function (test: both duplicates on the first position in ranked list)."
        
    # The second test: one candidate on the first position, another — on the second
    candidates_ranking = [["How Can I Make These Links Rotate in PHP", 
                           "How does the catch keyword determine the type of exception that was thrown"], 
                          ["Convert Google results object (pure js) to Python object",
                           "WPF- How to update the changes in list item of a list"]]
    dup_ranks = [candidates_ranking[i].index(answers[i]) + 1 for i in range(len(answers))]
    correct_answers = [0.5, 1]
    for k, correct in enumerate(correct_answers, 1):
        if not np.isclose(hits_count(dup_ranks, k), correct):
            return "Check the function (test: one candidate on the first position, another — on the second)."

    # The third test: both candidates on the second position
    candidates_ranking = [["How Can I Make These Links Rotate in PHP", 
                           "How does the catch keyword determine the type of exception that was thrown"], 
                          ["WPF- How to update the changes in list item of a list",
                           "Convert Google results object (pure js) to Python object"]]
    dup_ranks = [candidates_ranking[i].index(answers[i]) + 1 for i in range(len(answers))]
    correct_answers = [0, 1]
    for k, correct in enumerate(correct_answers, 1):
        if not np.isclose(hits_count(dup_ranks, k), correct):
            return "Check the function (test: both candidates on the second position)."

    return "Basic test are passed."

In [ ]:
print(test_hits())

Basic test are passed.


In [ ]:
def dcg_score(dup_ranks, k):
    """
        dup_ranks: list of duplicates' ranks; one rank per question; 
                   length is a number of questions which we are looking for duplicates; 
                   rank is a number from 1 to len(candidates of the question); 
                   e.g. [2, 3] means that the first duplicate has the rank 2, the second one — 3.
        k: number of top-ranked elements (k in DCG@k metric)

        result: return DCG@k value for current ranking
    """
    count = int(0)
    length = len(dup_ranks)

    for rank in dup_ranks:
      if(k>=rank):
        count+= (1/np.log2(1+rank))
      else:
        count+= 0 

    return count/length


In [ ]:
def test_dcg():
    # *Evaluation example*
    # answers — dup_i
    answers = ["How does the catch keyword determine the type of exception that was thrown"]
    
    # candidates_ranking — the ranked sentences provided by our model
    candidates_ranking = [["How Can I Make These Links Rotate in PHP", 
                           "How does the catch keyword determine the type of exception that was thrown",
                           "NSLog array description not memory address",
                           "PECL_HTTP not recognised php ubuntu"]]
    # dup_ranks — position of the dup_i in the list of ranks +1
    dup_ranks = [candidates_ranking[i].index(answers[i]) + 1 for i in range(len(answers))]
    
    # correct_answers — the expected values of the result for each k from 1 to 4
    correct_answers = [0, 1 / (np.log2(3)), 1 / (np.log2(3)), 1 / (np.log2(3))]
    for k, correct in enumerate(correct_answers, 1):
        if not np.isclose(dcg_score(dup_ranks, k), correct):
            return "Check the function."
    
    # Other tests
    answers = ["How does the catch keyword determine the type of exception that was thrown", 
               "Convert Google results object (pure js) to Python object"]

    # The first test: both duplicates on the first position in ranked list
    candidates_ranking = [["How does the catch keyword determine the type of exception that was thrown",
                           "How Can I Make These Links Rotate in PHP"], 
                          ["Convert Google results object (pure js) to Python object",
                           "WPF- How to update the changes in list item of a list"]]
    dup_ranks = [candidates_ranking[i].index(answers[i]) + 1 for i in range(len(answers))]
    correct_answers = [1, 1]
    for k, correct in enumerate(correct_answers, 1):
        if not np.isclose(dcg_score(dup_ranks, k), correct):
            return "Check the function (test: both duplicates on the first position in ranked list)."
        
    # The second test: one candidate on the first position, another — on the second
    candidates_ranking = [["How Can I Make These Links Rotate in PHP", 
                           "How does the catch keyword determine the type of exception that was thrown"], 
                          ["Convert Google results object (pure js) to Python object",
                           "WPF- How to update the changes in list item of a list"]]
    dup_ranks = [candidates_ranking[i].index(answers[i]) + 1 for i in range(len(answers))]
    correct_answers = [0.5, (1 + (1 / (np.log2(3)))) / 2]
    for k, correct in enumerate(correct_answers, 1):
        if not np.isclose(dcg_score(dup_ranks, k), correct):
            return "Check the function (test: one candidate on the first position, another — on the second)."
        
    # The third test: both candidates on the second position
    candidates_ranking = [["How Can I Make These Links Rotate in PHP",
                           "How does the catch keyword determine the type of exception that was thrown"], 
                          ["WPF- How to update the changes in list item of a list",
                           "Convert Google results object (pure js) to Python object"]]
    dup_ranks = [candidates_ranking[i].index(answers[i]) + 1 for i in range(len(answers))]
    correct_answers = [0, 1 / (np.log2(3))]
    for k, correct in enumerate(correct_answers, 1):
        if not np.isclose(dcg_score(dup_ranks, k), correct):
            return "Check the function (test: both candidates on the second position)."

    return "Basic test are passed."

In [ ]:
print(test_dcg())

Basic test are passed.


Submit results of the functions *hits_count* and *dcg_score* for the following examples to earn the points.

In [ ]:
test_examples = [
    [1],
    [1, 2],
    [2, 1],
    [1, 2, 3],
    [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    [9, 5, 4, 2, 8, 10, 7, 6, 1, 3],
    [4, 3, 5, 1, 9, 10, 7, 8, 2, 6],
    [5, 1, 7, 6, 2, 3, 8, 9, 10, 4],
    [6, 3, 1, 4, 7, 2, 9, 8, 10, 5],
    [10, 9, 8, 7, 6, 5, 4, 3, 2, 1],
]

In [ ]:
hits_results = []
for example in test_examples:
    for k in range(len(example)):
        hits_results.append(hits_count(example, k + 1))
grader.submit_tag('HitsCount', array_to_string(hits_results))

Current answer for task HitsCount is: 1.0
0.5
1.0
0.5
1.0
0.3333333333333333
0.6666666666666666
1.0
0.1
0.2
0.3
0.4
0.5
0.6
0.7
0.8
0.9
1....


In [ ]:
dcg_results = []
for example in test_examples:
    for k in range(len(example)):
        dcg_results.append(dcg_score(example, k + 1))
grader.submit_tag('DCGScore', array_to_string(dcg_results))

Current answer for task DCGScore is: 1.0
0.5
0.8154648767857288
0.5
0.8154648767857288
0.3333333333333333
0.5436432511904858
0.7103099178...


In [ ]:
# code-check
print(hits_results)

[1.0, 0.5, 1.0, 0.5, 1.0, 0.3333333333333333, 0.6666666666666666, 1.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]


In [ ]:
# code-check
print(dcg_results)

[1.0, 0.5, 0.8154648767857288, 0.5, 0.8154648767857288, 0.3333333333333333, 0.5436432511904858, 0.7103099178571526, 0.1, 0.16309297535714576, 0.21309297535714578, 0.25616063116448506, 0.29484591188793924, 0.3304666305987415, 0.3637999639320748, 0.3953464516106477, 0.4254494511770458, 0.4543559338088346, 0.1, 0.16309297535714576, 0.21309297535714578, 0.25616063116448506, 0.29484591188793924, 0.33046663059874143, 0.36379996393207475, 0.3953464516106477, 0.4254494511770458, 0.4543559338088346, 0.1, 0.16309297535714576, 0.21309297535714578, 0.25616063116448506, 0.29484591188793924, 0.3304666305987415, 0.36379996393207475, 0.39534645161064763, 0.42544945117704575, 0.4543559338088345, 0.1, 0.16309297535714576, 0.21309297535714578, 0.25616063116448506, 0.29484591188793924, 0.33046663059874143, 0.3637999639320747, 0.39534645161064763, 0.42544945117704575, 0.4543559338088345, 0.1, 0.16309297535714576, 0.21309297535714578, 0.25616063116448506, 0.29484591188793924, 0.33046663059874143, 0.36379996

##  First solution: pre-trained embeddings

We will work with predefined train, validation and test corpora. All the files are tab-separated, but have a different format:
 - *train* corpus contains similar sentences at the same row.
 - *validation* corpus contains the following columns: *question*, *similar question*, *negative example 1*, *negative example 2*, ... 
 - *test* corpus contains the following columns: *question*, *example 1*, *example 2*, ...

Validation corpus will be used for the intermediate validation of models. The test data will be necessary for submitting the quality of your model in the system.

Now you should read *validation* corpus, located at `data/validation.tsv`. You will use it later to evaluate current solution.

In [ ]:
def read_corpus(filename):
    data = []
    for line in open(filename, encoding='utf-8'):
        data.append(line.strip().split('\t'))
    return data

In [ ]:
validation = read_corpus('data/validation.tsv')

In [ ]:
print(f"LEngth of validation: {len(validation)}")
validation[:1]

LEngth of validation: 10000


[['How to print a binary heap tree without recursion?',
  'How do you best convert a recursive function to an iterative one?',
  'How can i use ng-model with directive in angular js',
  'flash: drawing and erasing',
  'toggle react component using hide show classname',
  'Use a usercontrol from another project to current webpage',
  '~ Paths resolved differently after upgrading to ASP.NET 4',
  'Materialize datepicker - Rendering when an icon is clicked',
  'Creating PyPi package - Could not find a version that satisfies the requirement iso8601',
  'How can I analyze a confusion matrix?',
  'How do I declare a C array in Swift?',
  'Using rand() when flipping a coin and rolling a die',
  'Handling a JSON field with a special character in its name in Java',
  'React Native select row on ListView when push it',
  "Get 'creation_time' of video using ffmpeg and regex",
  'Does row exist and multiple where',
  "How to specify a classifier in a gradle dependency's dependency?",
  'Using $unw

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

We will use cosine distance to rank candidate questions which you need to implement in the function *rank_candidates*. The function should return a sorted list of pairs *(initial position in candidates list, candidate)*. Index of some pair corresponds to its rank (the first is the best). For example, if the list of candidates was *[a, b, c]* and the most similar is *c*, then *a* and *b*, the function should return a list *[(2, c), (0, a), (1, b)]*.

Pay attention, if you use the function *cosine_similarity* from *sklearn.metrics.pairwise* to calculate similarity because it works in a different way: most similar objects has greatest similarity. It's preferable to use a vectorized version of *cosine_similarity* function. Try to compute similarity at once and not use list comprehension. It should speed up your computations significantly.

In [ ]:
def rank_candidates(question, candidates, embeddings, dim=300):
    """
        question: a string
        candidates: a list of strings (candidates) which we want to rank
        embeddings: some embeddings
        dim: dimension of the current embeddings
        
        result: a list of pairs (initial position in the list, question)
    """
    question_embeddings = np.zeros((1,dim))
    candidates_embeddings = np.zeros((len(candidates),dim))

    # Assign the question embeddings in the array form.
    question_embeddings[0] = question_to_vec(question, embeddings, dim)

    # Assigning the candidates embeddings in the array form with respect to question vectors.
    for j in range(len(candidates)):
      candidates_embeddings[j] = question_to_vec(candidates[j], embeddings, dim)
  
    result_idx = cosine_similarity(question_embeddings, candidates_embeddings).tolist()     # Convert into to_list
    result = [ (idx,k)  for idx,k in enumerate(result_idx[0]) ]    
    result.sort(key=lambda x:x[1], reverse=True)          
    final_result = [ (p[0], candidates[p[0]]) for p in result]
    return final_result


Test your code on the tiny examples:

In [ ]:
def test_rank_candidates():
    questions = ['converting string to list', 'Sending array via Ajax fails']
    candidates = [['Convert Google results object (pure js) to Python object', 
                   'C# create cookie from string and send it',
                   'How to use jQuery AJAX for an outside domain?'], 
                  ['Getting all list items of an unordered list in PHP', 
                   'WPF- How to update the changes in list item of a list', 
                   'select2 not displaying search results']]
    results = [[(1, 'C# create cookie from string and send it'), 
                (0, 'Convert Google results object (pure js) to Python object'), 
                (2, 'How to use jQuery AJAX for an outside domain?')],
               [(0, 'Getting all list items of an unordered list in PHP'), 
                (2, 'select2 not displaying search results'), 
                (1, 'WPF- How to update the changes in list item of a list')]]
    for question, q_candidates, result in zip(questions, candidates, results):
        ranks = rank_candidates(question, q_candidates, wv_embeddings, 300)
        if not np.all(ranks == result):
            return "Check the function."
    return "Basic tests are passed."

In [ ]:
print(test_rank_candidates())

Basic tests are passed.


Now we can test the quality of the current approach. Run the next two cells to get the results. Pay attention that calculation of similarity between vectors takes time and this calculation is computed approximately in 10 minutes.

In [ ]:
# Code-check 
for idx,line in enumerate(validation):
  m, *xo = line
  print(m)
  print(xo)
  print(line)
  if(idx==0):
    break

How to print a binary heap tree without recursion?
['How do you best convert a recursive function to an iterative one?', 'How can i use ng-model with directive in angular js', 'flash: drawing and erasing', 'toggle react component using hide show classname', 'Use a usercontrol from another project to current webpage', '~ Paths resolved differently after upgrading to ASP.NET 4', 'Materialize datepicker - Rendering when an icon is clicked', 'Creating PyPi package - Could not find a version that satisfies the requirement iso8601', 'How can I analyze a confusion matrix?', 'How do I declare a C array in Swift?', 'Using rand() when flipping a coin and rolling a die', 'Handling a JSON field with a special character in its name in Java', 'React Native select row on ListView when push it', "Get 'creation_time' of video using ffmpeg and regex", 'Does row exist and multiple where', "How to specify a classifier in a gradle dependency's dependency?", 'Using $unwind on multiple documents', 'Visual St

In [ ]:
wv_ranking = []
for line in validation:
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

In [ ]:
print(wv_ranking)

[8, 978, 4, 148, 27, 2, 19, 8, 1, 2, 433, 1, 990, 1, 2, 357, 579, 540, 811, 163, 1, 176, 1, 985, 1, 510, 910, 730, 381, 3, 207, 993, 344, 1, 23, 1, 1, 1, 46, 2, 1, 34, 158, 407, 177, 219, 1, 1, 262, 61, 1, 1, 43, 33, 15, 471, 504, 143, 85, 1, 2, 315, 1, 1, 31, 8, 12, 41, 395, 1, 6, 13, 189, 756, 61, 183, 1, 20, 278, 148, 622, 931, 1, 124, 123, 32, 1, 455, 49, 3, 1, 9, 3, 62, 70, 71, 31, 1, 254, 893, 43, 324, 617, 3, 1, 20, 166, 12, 28, 442, 2, 215, 2, 1, 1, 7, 35, 21, 1, 652, 319, 359, 105, 47, 953, 422, 1, 17, 9, 4, 87, 19, 334, 3, 50, 59, 9, 161, 4, 169, 299, 9, 611, 1, 1, 45, 780, 841, 1, 1, 61, 2, 414, 1, 371, 190, 240, 385, 5, 4, 88, 1, 21, 54, 1, 1, 9, 355, 2, 5, 1, 54, 1, 8, 8, 266, 891, 6, 2, 1, 2, 1, 218, 220, 1, 1, 285, 170, 14, 1, 818, 147, 1, 12, 732, 71, 13, 40, 1, 135, 1, 175, 801, 487, 66, 543, 26, 1, 125, 16, 167, 534, 369, 696, 907, 19, 42, 474, 502, 167, 964, 561, 2, 137, 1, 1, 56, 9, 1, 1, 792, 3, 1, 20, 34, 872, 349, 1, 1, 3, 13, 48, 24, 4, 166, 1, 734, 4, 31, 560, 

In [ ]:
for k in [1, 5, 10, 100, 500, 1000]:
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

DCG@   1: 0.217 | Hits@   1: 0.217
DCG@   5: 0.271 | Hits@   5: 0.319
DCG@  10: 0.288 | Hits@  10: 0.370
DCG@ 100: 0.326 | Hits@ 100: 0.560
DCG@ 500: 0.358 | Hits@ 500: 0.818
DCG@1000: 0.378 | Hits@1000: 1.000


If you did all the steps correctly, you should be frustrated by the received results. Let's try to understand why the quality is so low. First of all, when you work with some data it is necessary to have an idea how the data looks like. Print several questions from the data:

In [ ]:
for line in validation[:3]:
    q, *examples = line
    print(q)
    print(examples[:3])
    print("="*40)

How to print a binary heap tree without recursion?
['How do you best convert a recursive function to an iterative one?', 'How can i use ng-model with directive in angular js', 'flash: drawing and erasing']
How to start PhoneStateListener programmatically?
['PhoneStateListener and service', 'Java cast object[] to model', 'WCF and What does this mean?']
jQuery: Show a div2 when mousenter over div1 is over
['when hover on div1 depenting on if it is on div2 or not it should act differently', 'How to run selenium in google app engine/cloud?', 'Python Comparing two lists of strings for similarities']


As you can see, we deal with the raw data. It means that we have many punctuation marks, special characters and unlowercased letters. In our case, it could lead to the situation where we can't find some embeddings, e.g. for the word "grid?". 

To solve this problem you should use the functions *text_prepare* from the previous assignments to prepare the data.

In [ ]:
from util import text_prepare

Now transform all the questions from the validation set:

In [ ]:
from tqdm import tqdm

In [ ]:
prepared_validation = []
for line in tqdm(validation):               # validation here is list of list
  per_list = []                       # 
  for ques in line:
    clean_txt = text_prepare(ques)
    per_list.append(clean_txt)
  prepared_validation.append(per_list)

100%|██████████| 10000/10000 [00:46<00:00, 216.51it/s]


In [ ]:
# Code-check
print(len(prepared_validation))

for idx,x in enumerate(prepared_validation):
  print(x)
  if(idx==1):
    break


10000
['print binary heap tree without recursion', 'best convert recursive function iterative one', 'use ngmodel directive angular js', 'flash drawing erasing', 'toggle react component using hide show classname', 'use usercontrol another project current webpage', 'paths resolved differently upgrading aspnet 4', 'materialize datepicker rendering icon clicked', 'creating pypi package could find version satisfies requirement iso8601', 'analyze confusion matrix', 'declare c array swift', 'using rand flipping coin rolling die', 'handling json field special character name java', 'react native select row listview push', 'get creation_time video using ffmpeg regex', 'row exist multiple', 'specify classifier gradle dependencys dependency', 'using unwind multiple documents', 'visual studio code missing x509certificate2ui', 'developing cordova plugins firefox os', 'update multiple rows 2 columns mysql', 'analyse format alphanumeric string', 'dealing large tiff images c++ magick libtiff', 'error i

Let's evaluate the approach again after the preparation:

In [ ]:
wv_prepared_ranking = []
for line in prepared_validation:
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings)
    wv_prepared_ranking.append([r[0] for r in ranks].index(0) + 1)

In [ ]:
print(wv_prepared_ranking)

[41, 973, 2, 21, 11, 1, 1, 1, 1, 7, 170, 1, 987, 1, 1, 36, 306, 18, 750, 570, 1, 326, 1, 747, 1, 464, 23, 596, 486, 10, 670, 994, 66, 1, 16, 1, 1, 1, 48, 1, 1, 1, 253, 71, 5, 1, 1, 6, 70, 1, 2, 1, 282, 1, 1, 132, 293, 154, 437, 1, 1, 315, 1, 1, 1, 1, 10, 73, 1, 1, 4, 22, 7, 931, 149, 777, 1, 6, 98, 69, 90, 236, 1, 777, 22, 1, 1, 596, 1, 1, 1, 2, 1, 7, 2, 160, 14, 5, 1, 834, 499, 573, 636, 7, 1, 85, 187, 16, 1, 32, 1, 1, 3, 1, 1, 3, 37, 13, 1, 464, 272, 719, 281, 1, 851, 156, 77, 18, 3, 1, 52, 1, 780, 1, 32, 2, 1, 264, 2, 5, 396, 4, 465, 1, 1, 1, 903, 870, 3, 1, 1, 57, 312, 1, 508, 688, 216, 947, 1, 56, 1, 1, 188, 1, 1, 515, 3, 3, 1, 1, 1, 61, 432, 5, 6, 30, 812, 4, 1, 3, 1, 1, 416, 59, 1, 1, 11, 930, 487, 71, 744, 202, 1, 12, 766, 419, 2, 113, 1, 11, 1, 287, 616, 536, 30, 185, 18, 1, 2, 30, 2, 795, 141, 808, 771, 4, 1, 11, 1, 367, 731, 415, 2, 1, 1, 1, 28, 1, 1, 6, 475, 5, 1, 10, 1, 753, 409, 2, 3, 8, 16, 69, 1, 2, 104, 1, 230, 78, 45, 301, 1, 872, 1, 107, 38, 2, 3, 992, 1, 3, 6, 151, 

In [ ]:
for k in [1, 5, 10, 100, 500, 1000]:
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_prepared_ranking, k), 
                                              k, hits_count(wv_prepared_ranking, k)))

DCG@   1: 0.320 | Hits@   1: 0.320
DCG@   5: 0.387 | Hits@   5: 0.448
DCG@  10: 0.404 | Hits@  10: 0.500
DCG@ 100: 0.438 | Hits@ 100: 0.669
DCG@ 500: 0.460 | Hits@ 500: 0.844
DCG@1000: 0.477 | Hits@1000: 1.000


Now, prepare also train and test data, because you will need it in the future:

In [ ]:
# code-check
omg = open('data.tsv','w')
lop = ['dskbdkfk','knfjefnnk','fkjfkjefnkjewf','fjwefjefjke','kjefeJFKHFNWHUF','FJKBEFEFEBFJ' ]
print(*lop, sep='\t',file = omg)
# print(*lop)

In [ ]:
# code-check
for line in open('data.tsv', encoding='utf8'):
  print(line)

In [ ]:
def prepare_file(in_, out_):
    out = open(out_, 'w')
    for line in open(in_, encoding='utf8'):
        line = line.strip().split('\t')
        new_line = [text_prepare(q) for q in line]
        print(*new_line, sep='\t', file=out)
    out.close()

In [ ]:
prepare_file('data/train.tsv','clean_train.tsv')


In [ ]:
prepare_file('data/test.tsv', 'clean_test.tsv')


In [ ]:
# code-check
for idx,line in enumerate(open('clean_train.tsv')):
  print(line)
  if(idx==0):
    break

converting string list	convert google results object pure js python object



**Task 4 (W2VTokenizedRanks).** For each question from prepared *test.tsv* submit the ranks of the candidates to earn the points. The calculations should take about 3-5 minutes. Pay attention that the function *rank_candidates* returns a ranking, while in this case you should find a position in this ranking. Ranks should start with 1.

In [ ]:
from util import matrix_to_string

In [ ]:
w2v_ranks_results = []
prepared_test_data = 'data/clean_test.tsv'
for line in open(prepared_test_data):
    q, *ex = line.strip().split('\t')
    ranks = rank_candidates(q, ex, wv_embeddings, 300)
    ranked_candidates = [r[0] for r in ranks]
    w2v_ranks_results.append([ranked_candidates.index(i) + 1 for i in range(len(ranked_candidates))])
    
grader.submit_tag('W2VTokenizedRanks', matrix_to_string(w2v_ranks_results))

Current answer for task W2VTokenizedRanks is: 95	94	7	9	64	37	32	93	24	100	98	17	60	6	97	49	70	38	42	96	30	21	2	65	67	45	27	26	57	62	11	88	56	66	7...


## Advanced solution: StarSpace embeddings

Now you are ready to train your own word embeddings! In particular, you need to train embeddings specially for our task of duplicates detection. Unfortunately, StarSpace cannot be run on Windows and we recommend to use provided
[docker container](https://github.com/hse-aml/natural-language-processing/blob/master/Docker-tutorial.md) or other alternatives. Don't delete results of this task because you will need it in the final project.

### How it works and what's the main difference with word2vec?
The main point in this section is that StarSpace can be trained specifically for some tasks. In contrast to word2vec model, which tries to train similar embeddings for words in similar contexts, StarSpace uses embeddings for the whole sentence (just as a sum of embeddings of words and phrases). Despite the fact that in both cases we get word embeddings as a result of the training, StarSpace embeddings are trained using some supervised data, e.g. a set of similar sentence pairs, and thus they can better suit the task.

In our case, StarSpace should use two types of sentence pairs for training: "positive" and "negative". "Positive" examples are extracted from the train sample (duplicates, high similarity) and the "negative" examples are generated randomly (low similarity assumed). 

### How to choose the best params for the model?
Normally, you would start with some default choice and then run extensive experiments to compare different strategies. However, we have some recommendations ready for you to save your time:
- Be careful with choosing the suitable training mode. In this task we want to explore texts similarity which corresponds to *trainMode = 3*.
- Use adagrad optimization (parameter *adagrad = true*).
- Set the length of phrase equal to 1 (parameter *ngrams*), because we need embeddings only for words.
- Don't use a large number of *epochs* (we think that 5 should be enough).
- Try dimension *dim* equal to 100.
- To compare embeddings usually *cosine* *similarity* is used.
- Set *minCount* greater than 1 (for example, 2) if you don't want to get embeddings for extremely rare words.
- Parameter *verbose = true* could show you the progress of the training process.
- Set parameter *fileFormat* equals *labelDoc*.
- Parameter *negSearchLimit* is responsible for a number of negative examples which is used during the training. We think that 10 will be enought for this task.
- To increase a speed of training we recommend to set *learning rate* to 0.05.

Train StarSpace embeddings for unigrams on the train dataset. You don't need to change the format of the input data. Just don't forget to use prepared version of the training data. 

If you follow the instruction, the training process will take about 1 hour. The size of the embeddings' dictionary should be approximately 100 000 (number of lines in the result file). If you got significantly more than this number, try to check all the instructions above.

In [ ]:
######### TRAINING HAPPENING HERE #############
%%bash
./Starspace/starspace train -trainFile "data/clean_train.tsv" -model starspace_embedding \
-trainMode 3 -adagrad true -ngrams 1 -epoch 5 -dim 100 -similarity cosine -minCount 2 \
-verbose true -fileFormat labelDoc -negSearchLimit 10 -lr 0.05



Arguments: 
lr: 0.05
dim: 100
epoch: 5
maxTrainTime: 8640000
validationPatience: 10
saveEveryEpoch: 0
loss: hinge
margin: 0.05
similarity: cosine
maxNegSamples: 10
negSearchLimit: 10
batchSize: 5
thread: 10
minCount: 2
minCountLabel: 1
label: __label__
label: __label__
ngrams: 1
bucket: 2000000
adagrad: 1
trainMode: 3
fileFormat: labelDoc
normalizeText: 0
dropoutLHS: 0
dropoutRHS: 0
useWeight: 0
weightSep: :
Start to initialize starspace model.
Build dict from input file : data/clean_train.tsv
Loading data from file : data/clean_train.tsv
Total number of examples loaded : 999740
Initialized model weights. Model size :
matrix : 95058 100
Training epoch 0: 0.05 0.01

 ---+++                Epoch    0 Train error : 0.04388564 +++--- ☃
Training epoch 1: 0.04 0.01

 ---+++                Epoch    1 Train error : 0.01331890 +++--- ☃
Training epoch 2: 0.03 0.01

 ---+++                Epoch    2 Train error : 0.00942628 +++--- ☃
Training epoch 3: 0.02 0.01

 ---+++                Epoch    3 T

Read 12M words
Number of words in dictionary:  95058
Number of labels in dictionary: 0
Epoch: 100.0%  lr: 0.000060  loss: 0.006990  eta: <1min   tot: 0h7m49s  (100.0%)

And now we can compare the new embeddings with the previous ones. You can find trained word vectors in the file *[model_file_name].tsv*. Upload the embeddings from StarSpace into a dict. 

In [ ]:
starspace_embeddings = dict()                                     ######### YOUR CODE HERE #############

In [ ]:
!ls

boost_1_63_0			       setup_google_colab.py
boost_1_63_0.zip		       Starspace
boost_1_63_0.zip.1		       starspace_embedding
data				       starspace_embedding.tsv
GoogleNews-vectors-negative300.bin.gz  util.py
grader.py			       week3-Embeddings.ipynb
__pycache__


In [ ]:
# code-check to look for starspace words embeddings
for i,line in enumerate(open('starspace_embedding.tsv')):
  # print(line.split('\t'))
  check = line.strip()
  wrd, *embed = line.strip().split('\t')     # without using strip we have the newline character(\n) in the end of the list attached to 100-dim. 
  print(wrd)
  print((embed))
  print(check)

  if(i==0):
    break


using
['-0.0052222', '-0.00448992', '-0.00601377', '0.00856118', '0.00348905', '0.000855958', '-0.0222432', '-0.00448985', '0.0140598', '-0.0214769', '-0.0231381', '0.00106608', '0.0139505', '0.0143361', '-0.0221089', '0.0106953', '-0.00768639', '-0.00666279', '-0.0164317', '-0.0107159', '-0.000209061', '0.0286969', '-0.00139447', '-0.0164851', '-0.00526092', '0.00131748', '-0.0030131', '-0.00562873', '0.0133654', '-0.0149031', '0.0144998', '0.00638129', '0.0061422', '0.0260459', '-0.0166875', '0.00738028', '0.0199978', '0.0151107', '-0.0152152', '0.00929968', '-0.00424503', '-0.00623289', '-0.0335389', '-0.00533453', '0.00175515', '0.0248907', '0.0160316', '0.00500873', '-0.00131908', '-0.00723413', '0.0123936', '-0.00259814', '0.0182779', '0.0171229', '-0.000521789', '-0.00938466', '0.0173563', '0.00771665', '-0.0164065', '-0.00660952', '-0.00435235', '-0.00152252', '-0.00378012', '-0.00928007', '-0.0316654', '-0.002826', '0.0141284', '-0.0390664', '-0.000340338', '-0.00386237', '-0.

In [ ]:
starspace_embeddings = dict()                                     ######### YOUR CODE HERE #############
for line in open('starspace_embedding.tsv'):
  wrd, *embedding = line.strip().split('\t')
  starspace_embeddings[wrd] = np.array(embedding, dtype=np.float32)


In [ ]:
# code-check
# print(starspace_embeddings)
print(len(starspace_embeddings.keys()))

95058


In [ ]:
ss_prepared_ranking = []
for line in prepared_validation:
    q, *ex = line
    ranks = rank_candidates(q, ex, starspace_embeddings, dim=100)
    ss_prepared_ranking.append([r[0] for r in ranks].index(0) + 1)

In [ ]:
print(len(ss_prepared_ranking))
print(ss_prepared_ranking)

10000
[1, 1, 1, 1, 1, 1, 1, 4, 1, 3, 1, 1, 38, 1, 1, 1, 2, 11, 113, 3, 1, 126, 1, 147, 1, 1, 3, 1, 1, 8, 5, 17, 101, 1, 3, 2, 1, 1, 23, 1, 1, 24, 1, 1, 3, 1, 1, 2, 37, 1, 1, 1, 72, 1, 1, 1, 6, 22, 1, 1, 1, 1, 1, 1, 2, 1, 3, 1, 1, 1, 4, 99, 1, 55, 212, 394, 2, 3, 134, 2, 1, 355, 1, 2, 2, 1, 1, 280, 1, 2, 1, 6, 1, 3, 1, 70, 1, 3, 1, 5, 53, 20, 39, 3, 1, 19, 12, 38, 1, 35, 2, 5, 1, 1, 1, 1, 1, 88, 1, 14, 46, 603, 37, 4, 6, 1, 1, 1, 1, 1, 1, 1, 8, 1, 5, 1, 1, 1, 2, 4, 2, 8, 47, 1, 1, 1, 1, 189, 5, 1, 1, 2, 220, 1, 1, 1, 107, 1, 1, 1, 2, 1, 5, 2, 1, 1, 1, 1, 1, 3, 1, 2, 1, 1, 7, 1, 749, 5, 10, 1, 1, 1, 41, 1, 6, 1, 33, 4, 445, 9, 382, 1, 1, 19, 27, 690, 9, 189, 1, 179, 1, 24, 1, 1, 99, 23, 1, 1, 1, 4, 3, 1, 3, 122, 908, 2, 1, 1, 1, 9, 71, 18, 1, 1, 1, 1, 18, 1, 1, 1, 3, 2, 1, 1, 1, 1, 11, 1, 1, 1, 13, 1, 1, 1, 69, 8, 153, 1, 10, 2, 1, 30, 1, 2, 160, 2, 1, 1, 1, 1, 677, 8, 86, 1, 32, 7, 1, 1, 1, 1, 2, 1, 39, 1, 160, 1, 1, 1, 1, 1, 2, 1, 35, 1, 1, 1, 12, 1, 168, 67, 3, 10, 5, 474, 1, 2, 1, 77

In [ ]:
for k in [1, 5, 10, 100, 500, 1000]:
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(ss_prepared_ranking, k), 
                                               k, hits_count(ss_prepared_ranking, k)))

DCG@   1: 0.515 | Hits@   1: 0.515
DCG@   5: 0.611 | Hits@   5: 0.693
DCG@  10: 0.630 | Hits@  10: 0.753
DCG@ 100: 0.662 | Hits@ 100: 0.905
DCG@ 500: 0.671 | Hits@ 500: 0.979
DCG@1000: 0.674 | Hits@1000: 1.000


Due to training for the particular task with the supervised data, you should expect to obtain a higher quality than for the previous approach. In additiion, despite the fact that StarSpace's trained vectors have a smaller dimension than word2vec's, it provides better results in this task.

**Task 5 (StarSpaceRanks).** For each question from prepared *test.tsv* submit the ranks of the candidates for trained representation.

In [ ]:
starspace_ranks_results = []
prepared_test_data = 'data/clean_test.tsv'                                                       ######### YOUR CODE HERE #############
for line in open(prepared_test_data):
    q, *ex = line.strip().split('\t')
    ranks = rank_candidates(q, ex, starspace_embeddings, 100)
    ranked_candidates = [r[0] for r in ranks]
    starspace_ranks_results.append([ranked_candidates.index(i) + 1 for i in range(len(ranked_candidates))])
    
grader.submit_tag('StarSpaceRanks', matrix_to_string(starspace_ranks_results))

Current answer for task StarSpaceRanks is: 96	53	63	29	30	67	85	45	50	55	80	33	40	38	100	94	6	64	74	36	87	47	16	31	81	44	51	82	86	2	24	21	93	18...


Please, **don't remove** the file with these embeddings because you will need them in the final project.

### Authorization & Submission
To submit assignment parts to Cousera platform, please, enter your e-mail and token into variables below. You can generate token on this programming assignment page. <b>Note:</b> Token expires 30 minutes after generation.

In [ ]:
STUDENT_EMAIL = 'nikhil15.rana@gmail.com' 
STUDENT_TOKEN = 'xmdewRjxe4ApXmSz'
grader.status()

You want to submit these parts:
Task Question2Vec: 0.019293891059027776
-0.028727213541666668
0.046056111653645836
0.08525933159722222
0.02430555555555...
Task HitsCount: 1.0
0.5
1.0
0.5
1.0
0.3333333333333333
0.6666666666666666
1.0
0.1
0.2
0.3
0.4
0.5
0.6
0.7
0.8
0.9
1....
Task DCGScore: 1.0
0.5
0.8154648767857288
0.5
0.8154648767857288
0.3333333333333333
0.5436432511904858
0.7103099178...
Task W2VTokenizedRanks: 95	94	7	9	64	37	32	93	24	100	98	17	60	6	97	49	70	38	42	96	30	21	2	65	67	45	27	26	57	62	11	88	56	66	7...
Task StarSpaceRanks: 96	53	63	29	30	67	85	45	50	55	80	33	40	38	100	94	6	64	74	36	87	47	16	31	81	44	51	82	86	2	24	21	93	18...


If you want to submit these answers, run cell below

In [ ]:
grader.submit(STUDENT_EMAIL, STUDENT_TOKEN)

Submitted to Coursera platform. See results on assignment page!
